# Title

First we have to import necessary libraries.

In [ ]:
import math
import numpy as np
from scipy import linalg
from scipy import integrate
from scipy import sparse
import scipy.optimize as optimize
import qutip as qt
import os
#from EDAspy.optimization import UMDAd
import random
import matplotlib.pyplot as plt
import itertools

We define functions for spin operators.

In [ ]:
# Dictionary mapping operator specifications to the corresponding operator functions
opstr2fun = {'x': lambda dim: qt.spin_Jx((dim-1)/2),
             'y': lambda dim: qt.spin_Jy((dim-1)/2),
             'z': lambda dim: qt.spin_Jz((dim-1)/2),
             'p': lambda dim: qt.spin_Jp((dim-1)/2),
             'm': lambda dim: qt.spin_Jm((dim-1)/2),
             'i': qt.identity}

def mkSpinOp(dims, specs):
    """Returns a tensor product of operators based on given dimensions and specifications.
    
    Parameters
    ----------
    dims : list of int
        List of dimensions of individual operators.
    specs :
        List of index-specification pairs indicating which operator to modify and how.

    Returns
    -------
    qobj
        The matrix (tensor product of individual operators).
        
    Examples
    --------
    >>> spin_op = mkSpinOp([3, 4, 2], [(0, 'x'), (1, 'y'), (2, 'z')])
    >>> spin_op
    [[3, 4, 2], [3, 4, 2]]
    """

    ops = [qt.identity(d) for d in dims]
    for ind, opstr in specs:
        ops[ind] = ops[ind] * opstr2fun[opstr](dims[ind])
    return qt.tensor(ops)

def idOp(dims):
    """Returns the tensor product of identity operators with given dimensions.
    
    Parameters
    ----------
    dims : list of int
        List of dimensions of individual operators.

    Returns
    -------
    qobj
        The identity matrix (tensor product of individual operators).
        
    Examples
    --------
    >>> id_op = idOp([3, 4, 2])
    >>> id_op.dims
    [[3, 4, 2], [3, 4, 2]]
    """

    return mkSpinOp(dims, [])

def zeroOp(dims):
    """Returns the tensor product of zero operators with given dimensions.

    Parameters
    ----------
    dims : list of int
        List of dimensions of individual operators.

    Returns
    -------
    qobj
        The zero matrix (tensor product of individual operators).
        
    Examples
    --------
    >>> zero_op = zeroOp([3, 4, 2])
    >>> zero_op.dims
    [[3, 4, 2], [3, 4, 2]]
    """

    d = np.prod(dims)
    return qt.Qobj(sparse.csr_matrix((d, d), dtype=np.float64), 
                       dims=[list(dims)]*2, type="oper", isherm=True) 

We define functions for Hamiltonian operators.

In [ ]:
def mkH1(dims, ind, parvec):
    """Returns the Hamiltonian operator based on the given dimensions, index, and parameter vector.

    Parameters
    ----------
    dims : list of int
        List of dimensions of individual operators.
    ind : int
        Index indicating which operator to modify.
    parvec : list of float
        List of parameter values for each spin component. (???)

    Returns
    -------
    qutip.qobj
        The Hamiltonian operator.
        
    Examples
    --------
    >>> H1 = mkH1([3, 4, 2], 1, [1.0, 0.5, 0.1])
    >>> H1.dims
    [[3, 4, 2], [3, 4, 2]]
    """

    axes = ['x', 'y', 'z']
    components = [v * mkSpinOp(dims, [(ind,ax)]) for v, ax in zip(parvec, axes) if v!=0]
    if components:
        return sum(components)
    else:
        return zeroOp(dims)

def mkH12(dims, ind1, ind2, parmat):
    """Returns the Hamiltonian operator based on the given dimensions, indices, and parameter matrix.
    
    Parameters
    ----------
    dims : list of int
        List of dimensions of individual operators.
    ind1 : int
        Index indicating the first operator to modify.
    ind2 : int
        Index indicating the second operator to modify.
    parmat : np.ndarray
        Parameter matrix specifying the coupling strengths. (???)

    Returns
    -------
    qutip.qobj
        The Hamiltonian operator.
        
    Examples
    --------
    >>> H12 = mkH12([3, 4, 2], 0, 1, np.array([[1.0, 0.5, -0.3], [0.5, 0.0, 0.2], [-0.3, 0.2, 0.7]]))
    >>> H12.dims
    [[3, 4, 2], [3, 4, 2]]
    """
    
    axes = ['x', 'y', 'z']
    components = []
    for i in range(3):
        for j in range(3):
            if parmat[i,j] != 0:
                components.append(parmat[i,j] * mkSpinOp(dims, [(ind1,axes[i]), (ind2,axes[j])]))
    if components:
        return sum(components)
    else:
        return zeroOp(dims)

We define function for the dense of an operator.

In [ ]:
def dense(op):
    """Returns the dense array representation of the given operator.

    Parameters
    ----------
    op : Union[qutip.qobj.Qobj, scipy.sparse.csc_matrix]
        The input operator.

    Returns
    -------
    numpy.ndarray
        The dense array representation of the operator.
        
    Examples
    --------
    >>> operator = qt.Qobj([[1, 2], [3, 4]])
    
    # Get the dense array representation of the operator
    >>> dense_op = dense(operator)
    array([[1.+0.j, 2.+0.j], 
        [3.+0.j, 4.+0.j]])
    """
    
    if type(op) is qt.qobj.Qobj:
        return np.asarray(op.data.todense())
    else:
        return np.asarray(op.todense())

$$
\text{Trace}({\rho(t) Z})
$$

for 

$$
\rho(t) = U(t)\rho(0) U^{\dagger}(t)
$$

We define functions for computing expectation values.

In [ ]:
def expvalsConstH(Heff, rho0, ops, dt, nr_steps):
    """Returns the expectation values of operators and the final density matrix.

    Parameters
    ----------
    Heff : numpy.ndarray
        The effective Hamiltonian.
    rho0 : numpy.ndarray
        The initial density matrix.
    ops : list of numpy.ndarray
        List of operators for which expectation values are computed.
    dt : float
        The time step size.
    nr_steps : int
        The number of steps in the evolution.

    Returns
    -------
    obs : numpy.ndarray
        The expectation values of the operators over time.
    rhot : numpy.ndarray
        The final density matrix.
        
    Examples
    --------
    >>> Heff = np.array([[1, 0], [0, -1]])
    >>> rho0 = np.array([[1, 0], [0, 0]])
    >>> ops = [np.array([[1, 0], [0, 1]]), np.array([[0, 1], [1, 0]])]
    >>> dt = 0.1
    >>> nr_steps = 10

    # Compute the expectation values and final density matrix
    >>> obs, rhot = expvalsConstH(Heff, rho0, ops, dt, nr_steps)
    >>> obs
    array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
    >>> rhot
    array([[1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j]])
    """

    obs = np.zeros((len(ops),nr_steps))
    rhot = rho0
    U = linalg.expm(-1j*dt*Heff)
    for i in range(nr_steps):
        rhot = U @ rhot @ U.T.conjugate()
        for s in range(len(ops)):
            obs[s,i] = np.real(np.trace(ops[s] @ rhot))
                
    return obs, rhot

def expvalsPiecewiseConstHt(H0, H1, u, rho0, ops, dt):
    """Returns the expectation values of operators and the final density matrix.

    Parameters
    ----------
    H0 : numpy.ndarray
        The initial fixed Hamiltonian.
    H1 : numpy.ndarray
        The drift (control) time-dependent Hamiltonian.
    u : numpy.ndarray
        The magnetic field (dependent variable).
    rho0 : numpy.ndarray
        The initial density matrix.
    ops : list of numpy.ndarray
        List of operators for which expectation values are computed.
    dt : float
        The time step size.
    
    Returns
    -------
    obs : numpy.ndarray
        The expectation values of the operators over time.
    rhot : numpy.ndarray
        The final density matrix.
        
    Examples
    --------
    >>> H0 = np.array([[1, 0], [0, -1]])
    >>> H1 = np.array([[0, 1], [1, 0]])
    >>> u = np.array([0.1, 0.2, 0.3])
    >>> rho0 = np.array([[1, 0], [0, 0]])
    >>> ops = [np.array([[1, 0], [0, 1]]), np.array([[0, 1], [1, 0]])]
    >>> dt = 0.01

    # Compute the expectation values and final density matrix
    >>> obs, rhot = expvalsPiecewiseConstHt(H0, H1, u, rho0, ops, dt)
    >>> obs
    array([[1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       [1.99993267e-05, 9.99883939e-05, 2.79931701e-04]])
    >>> rhot
    array([[9.99964010e-01+1.73472348e-18j, 1.39965851e-04+5.99745641e-03j],
       [1.39965851e-04-5.99745641e-03j, 3.59903691e-05+0.00000000e+00j]])
    """

    nr_steps = len(u)

    obs = np.zeros((len(ops),nr_steps))
    rhot = rho0
    for i in range(nr_steps):
        Hi = H0 + u[i] * H1
        Ui = linalg.expm(-1j*dt*Hi)
        rhot = Ui @ rhot @ Ui.T.conjugate()
        for s in range(len(ops)):
            obs[s,i] = np.real(np.trace(ops[s] @ rhot))
                
    return obs, rhot

In [ ]:
# exp(iθn⋅σ) = cosθ I+i(n⋅σ)sinθ
# exp(-iudt Sx) = exp(-iudt/2 σx) = cos(udt/2) I - iσx sin(udt/2)
def expm_trotter_H1x(expiA, b1, u, ds, nr_halvings):
    """Returns the approximate exponential of a Hamiltonian.

    Parameters
    ----------
    expiA : numpy.ndarray
        The exponential of the A Hamiltonian.
    b1 : XXX
        xxx
    u : numpy.ndarray
        The magnetic field (dependent variable)
    ds : int
        Time step size.
    nr_halvings : int
        The number of halvings.

    Returns
    -------
    U : numpy.ndarray
        The approximate exponential of the Hamiltonian.
        
    Examples
    --------
    >>> expiA = np.eye(4)
    >>> b1 = 0.1
    >>> u = 0.1
    >>> ds = 0.01
    >>> nr_halvings = 3

    # Compute the approximate exponential of the Hamiltonian
    >>> U = expm_trotter_H1x(expiA, b1, u, ds, nr_halvings)
    """

    dim_n = expiA.shape[0]//4
    omega = b1*u*ds/2
    c = np.cos(omega)
    s = -1j * np.sin(omega)
    expiB_1 = np.array([[c,s],[s,c]])
    dexpiB_1 = (-1j*ds*b1/2) * np.array([[s,c],[c,s]])
    expiB_12 = np.kron(expiB_1, expiB_1)
    dexpiB_12 = np.kron(dexpiB_1, expiB_1) + np.kron(expiB_1, dexpiB_1)
    # expiB = sparse.kron(expiB_12, sparse.eye(dim_n, format="csr"))
    # dexpiB = sparse.kron(dexpiB_12, sparse.eye(dim_n, format="csr"))
    # expiB = sparse.kron(sparse.csr_matrix(expiB_12), qt.fastsparse.fast_identity(dim_n))
    # dexpiB = sparse.kron(sparse.csr_matrix(dexpiB_12), qt.fastsparse.fast_identity(dim_n))
    expiB = np.kron(expiB_12, np.eye(dim_n))
    dexpiB = np.kron(dexpiB_12, np.eye(dim_n))
    U = expiA @ (expiB @ expiA)
    L = expiA @ (dexpiB @ expiA)
    for i in range(nr_halvings, 0, -1):
        L = U @ L + L @ U
        U = U @ U
    return U

### Gradient functions Eduardo: 

In [ ]:
def expvalGradMultiSamples2(H0, H1, u, rho0, op, dt, w):
    
    nr_samples = len(w)
    nr_steps = len(u)
#    assert(np.mod(nr_steps, nr_samples) == 0)
    nr_substeps = nr_steps//nr_samples
    
    U_list = []
    for i in range(nr_steps):
        Hi = H0 + u[i] * H1
        Ui, _ = linalg.expm_frechet(-1j*dt*Hi, -1j*dt*H1) #GRADIENT HERE
        U_list.append(Ui)
        
    obs = 0.0
        
    Uf = np.eye(H0.shape[0]) # after loop: UnUn-1...U1U0
#    dUUf_list = [] # GRADIENT HERE
#    Ub_list = []   # GRADIENT HERE
    for j in range(nr_samples):
        m = j * nr_substeps
        n = m + nr_substeps
        Ub_list_j = []
        Ub = np.eye(H0.shape[0]) # after loop: UnUn-1...Um
        for i in range(nr_substeps):
            Ui = U_list[i + m] #GRADIENT HERE?
#            dUUf_list.append(dUi @ Uf) #GRADIENT HERE?
            Uf = Ui @ Uf
            Ub_list_j.insert(0, Ub) #GRADIENT HERE?
            Ub = Ub @ U_list[n-1-i][0]
#        for i in range(len(Ub_list)):
#            Ub_list[i] = Ub @ Ub_list[i]
#        Ub_list = Ub_list + Ub_list_j
        A = rho0 @ Uf.T.conjugate() 
        #print(A.shape)
        B = A @ op
        BT = B.T
        # obs += w[j] * np.real(np.trace(Uf @ B))
        obs += w[j] * np.real(np.sum(Uf.__mul__(BT)))
            
    rhot = Uf @ A
        
    return obs, Uf


def expvalGradMultiSamplesSplitTrotterH1x2(H0, b1, u, rho0, op, dt, w, nr_halvings=4):
    
    nr_samples = len(w)
    nr_steps = len(u)
#    assert(np.mod(nr_steps, nr_samples) == 0)
    nr_substeps = nr_steps//nr_samples
    
    dim = rho0.shape[0]
    
    ds = dt / 2**nr_halvings
    UA = linalg.expm(-1j*ds/2*H0) 
    
    U_list = []
    for i in range(nr_steps):
        # Hi = H0 + u[i] * H1 
        # Ui, dUi = linalg.expm_frechet(-1j*dt*Hi, -1j*dt*H1)
        Ui, _ = expm_trotter_H1x(UA, b1, u[i], ds, nr_halvings)
        U_list.append((Ui,dUi))
        
    obs = 0.0
        
    Uf = np.eye(dim) # after loop: UnUn-1...U1U0
#    dUUf_list = [] # [dU0, dU1U0, dU2U1U0, dU3U2U1U0, ..., dUnUn-1...U1U0] #GRADIENT HERE
    Ub_list = []   # [Un...U2U1, Un...U2, ..., UnUn-1Un-2, UnUn-1, Un, 1] #GRADIENT HERE
    for j in range(nr_samples):
        m = j * nr_substeps
        n = m + nr_substeps
        Ub_list_j = []
        Ub = np.eye(dim) # after loop: UnUn-1...Um
        for i in range(nr_substeps):
            Ui, dUi = U_list[i + m]
#            dUUf_list.append(dUi @ Uf) #GRADIENT HERE
            Uf = Ui @ Uf
            Ub_list_j.insert(0, Ub)
            Ub = Ub @ U_list[n-1-i][0]
        for i in range(len(Ub_list)):
            Ub_list[i] = Ub @ Ub_list[i]
        Ub_list = Ub_list + Ub_list_j
        A = rho0 @ Uf.T.conjugate()
        #print(A.shape)
        B = A @ op
        BT = B.T
        # obs += w[j] * np.real(np.trace(Uf @ B))
        obs += w[j] * np.real(np.sum(Uf.__mul__(BT)))
        
            
    rhot = Uf @ A
        
    return obs, Uf

In [ ]:
def fun2(u, H0, H1, rho0, obs, dt, w, minimize):
    if type(H1) == float:
        ys,_ = expvalGradMultiSamplesSplitTrotterH1x2(H0, H1, u, rho0, obs, dt, w)
    else:
        ys,_ = expvalGradMultiSamples2(H0, H1, u, rho0, obs, dt, w)
    if not minimize:
        ys = -ys
        #grad = -grad
           
    return ys

We look at a generalisation of the system studied in this paper: https://aip.scitation.org/doi/abs/10.1063/1.5131557

In [ ]:
def mkSystem(b0, b1, kS, k0):

    g = 2.00231930436256 #electron g-factor
    beta = 9.274009994e-24 #bohr magneton
    hbar = 6.62607015e-34/(2*np.pi) #hbar
    mT = g*beta/hbar*1e-9 # mT -> Mrad/s

    omega0 = b0*mT
    omega1 = b1*mT
    Is = [0.5, 0.5, 0.5, 0.5, 0.5]
    indE = [0, 0, 0, 1, 1]
    hfcs = np.array([0.2, 0.5, 1.0, 0.2, 0.3]) * mT # mT -> Mrad/s
    jex = 1 * 2*math.pi

    dims = [2, 2] + [round(2*Is[i]+1) for i in range(len(Is))]
    Hhfc = sum(mkH12(dims, indE[i], i+2, np.eye(3)*hfcs[i]) for i in range(len(hfcs)))
    Hzee = mkH1(dims, 0, [0,0,omega0]) + mkH1(dims, 1, [0,0,omega0])
    Hex = -jex * (1/2*mkSpinOp(dims, []) + mkH12(dims, 0, 1, 2*np.identity(3)))
    H0 = Hhfc + Hzee + Hex
    H1 = mkH1(dims, 0, [omega1,0,0]) + mkH1(dims, 1, [omega1,0,0])
    Ps = 0.25 * mkSpinOp(dims,[]) - mkH12(dims, 0, 1, np.eye(3))
    H = H0 - 1j * kS/2 * Ps
    rho0 = Ps/Ps.tr()
    H, H1, rho0 = [dense(op) for op in (H, H1, rho0)]
    Ps = Ps.data

    sys = {
        'Is': Is,
        'indE': indE,
        'hfcs': hfcs,
        'jex': jex,
        'omega0': omega0,
        'omega1': omega1,
        'kS': kS,
        'k0': k0
    }

    return H, H1, omega1, rho0, Ps, sys

In [ ]:
def optimizeYield(name, b0, b1, kS, k0, nr_cycles, nr_steps, nr_samples, eda_opt):

    # Optimize singlet probability after nr_steps steps of dt usind L-BFGS-B

    dt = 0.001
    nr_total = max(math.ceil(5/k0) * 1000, nr_cycles*nr_steps)
    nr_iter = 200
    minimize = True

    t_sampled = (np.arange(nr_samples) + 1) * (nr_steps//nr_samples)*dt # without t0
    # w = np.ones(nr_samples)
    w = np.exp(-t_sampled*k0) * k0 * (nr_steps//nr_samples)*dt

    H, H1, omega1, rho0, Ps, sys = mkSystem(b0, b1, kS, k0)

    rho = rho0
    
    #stats = []
    uopt_all = []
    for i in range(nr_cycles):
        u0 = np.random.randn(nr_steps)
        ind = np.where(np.abs(u0) > 1)[0]
        u0[ind] = np.sign(u0[ind])
        
        if eda_opt:
            eda = UMDAd(size_gen=30, max_iter=100, dead_iter=10, n_variables=8, alpha=0.5, vector=None,
            lower_bound=0.2, upper_bound=0.9, elite_factor=0.2, disp=True)

            eda_result = eda.minimize(cost_function=fun2(u0, H, H1, rho0, obs, dt, w, minimize), output_runtime=True)
            
            """
            CHANGE THIS LINE ABOVE WITH THE CORRECT OBS!!!!!
            """
            
            #eda_result = eda.minimize(cost_function=fun2(u0, H, H1, rho0, obs = np.eye(128), dt, w, minimize),
            #                          output_runtime= True)
        else:
            opt = optimize.minimize(fun, u0, 
                        args=(H, omega1, rho, Ps, dt, w, minimize),
                        method='L-BFGS-B',
                        jac=True, bounds=[(-1,1) for _ in range(len(u0))],
                        options={'disp': 0, 'maxcor': 60, 'ftol': 1e-6, 'gtol': 1e-7, 'maxiter': nr_iter})
            print(opt.message)
            print(opt.fun)
        
        
        u = opt.x
        uopt_all.append(u)
        #stats.append((opt.fun, opt.nit, opt.nfev, opt.success))
        
        ps, rho = expvalsPiecewiseConstHt(H, H1, u, rho, [Ps], dt)
        
    u = np.concatenate(uopt_all)
    u = np.concatenate((u, np.zeros(nr_total-len(u))))
    t = np.arange(len(u))*dt
    ps, _ = expvalsPiecewiseConstHt(H, H1, u, rho0, [Ps], dt)
    ps = ps[0]
    ys = integrate.simps(ps*k0*np.exp(-k0*t))*dt
    ps0, _ = expvalsConstH(H, rho0, [Ps], dt, nr_total)
    ps0 = ps0[0]
    ys0 = integrate.simps(ps0*k0*np.exp(-k0*t))*dt
    print("{} x {} @ {} : {} vs. {}".format(nr_cycles, nr_steps, nr_samples, ys, ys0))
    
    np.savez(name, sys=sys, u=u, 
             nr_cycles=nr_cycles, nr_steps=nr_steps, nr_samples=nr_samples, nr_iter=nr_iter, minimize=minimize,
             ys=ys, ys0=ys0, ps=ps, ps0=ps0) #stats=stats)

    return ys, ps #, stats

## Now we train normally! 

In [ ]:
nr_cycles = 1       # number of cycles
nr_steps = 1000     # number of steps
dt = 0.001          # time step
nr_samples = 100    # number of samples

b0 = 0.05 # mT (this is the geomagnetic field, relavant to avian magnetoreception)
b1 = 0.25 # mT, free to try different parameters here
k0 = 1.0 # 1/us
kS = 1.0 # 1/us

nr_total = max(math.ceil(5/k0) * 1000, nr_cycles*nr_steps)
nr_iter = 200
minimize = True     # are we using minimalization or maximalization function?

t_sampled = (np.arange(nr_samples) + 1) * (nr_steps//nr_samples)*dt # without t0
# w = np.ones(nr_samples)
w = np.exp(-t_sampled*k0) * k0 * (nr_steps//nr_samples)*dt

H, H1, omega1, rho0, Ps, sys = mkSystem(b0, b1, kS, k0)

rho = rho0

#obs = np.eye(128,128)
obs = Ps


for i in range(100):
    u0 = np.random.randn(100)
    ind = np.where(np.abs(u0) > 1)[0]
    u0[ind] = np.sign(u0[ind])

args = (H, H1, rho0, obs, dt, w, minimize)
fun2(u0, *args)

Reinforcement learning

In [ ]:
from collections import deque
import torch as T
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.auto import tqdm

define neural network

In [ ]:
class Network(nn.Module):
    def __init__(self, state_dim, hidden_dim1, hidden_dim2, action_dim):
        super(Network, self).__init__()
        self.fc1 = nn.Linear(state_dim, hidden_dim1)
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.fc3 = nn.Linear(hidden_dim2, action_dim)

    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        actions = self.fc3(x)
        return actions

define agent

In [ ]:
class Agent:
    def __init__(self, gamma, epsilon, tau, lr, state_dim, hidden_dim1, hidden_dim2, action_dim, device):
        self.device = device
        self.gamma = gamma  # discount fator
        self.epsilon = epsilon  # exploration rate
        self.tau = tau # target network update frequency
        self.lr = lr # learning rate
        self.action_dim = action_dim
        self.memory = deque(maxlen=10000)
        self.q_net = Network(state_dim, hidden_dim1, hidden_dim2, action_dim).to(self.device) # policy network
        self.target_q_net = Network(state_dim, hidden_dim1, hidden_dim2, action_dim).to(self.device) # target network
        self.optimizer = optim.Adam(self.q_net.parameters(), lr=self.lr)
        self.count = 0

    def memorize(self, state, action, reward, next_state):
        """
        store (state, action, reward, next_state) for experience replay
        """
        self.memory.append((state, action, reward, next_state)) 

    def sample(self, batch_size):
        """
        sample (state, action, reward, next_state) from memory
        """
        samples = random.sample(self.memory, batch_size)
        states, actions, rewards, next_states = zip(*samples)
        return np.array(states), np.array(actions), np.array(rewards), np.array(next_states)

    def take_action(self, state):
        if np.random.random() < self.epsilon:
            action = np.random.randint(self.action_dim)
        else:
            state = T.tensor([state], dtype=T.float).to(self.device)
            action = self.q_net(state).argmax().item()
        return action

    def update(self, states, actions, rewards, next_states, done):
        """
        update the target network
        """
        states = T.tensor(states, dtype=T.float).to(self.device)
        actions = T.tensor(actions).view(-1, 1).to(self.device)
        rewards = T.tensor(rewards, dtype=T.float).view(-1, 1).to(self.device)
        next_states = T.tensor(next_states, dtype=T.float).to(self.device)

        q = self.q_net(states).gather(1, actions)  # Q-value
        next_q = self.target_q_net(next_states).max(1)[0].view(-1, 1) # max Q-value of the next state
        q_targets = rewards + self.gamma * next_q * (1 - done) # expected Q-value
        loss = T.mean(F.mse_loss(q, q_targets))
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        if self.count % self.tau == 0:
            self.target_q_net.load_state_dict(self.q_net.state_dict())
        self.count += 1

In [ ]:
num_episodes = 50
gamma = 0.95
epsilon = 0.01
tau = 10
lr = 1e-3
state_dim = 100
hidden_dim1 = 64
hidden_dim2 = 64
action_dim = 21
batch_size = 48
device = T.device("cuda") if T.cuda.is_available() else T.device("cpu")
agent = Agent(gamma, epsilon, tau, lr, state_dim, hidden_dim1, hidden_dim2, action_dim, device)

episode_list = []
yield_list = []
state_list = []
for e in tqdm(range(num_episodes), desc="Main", leave=True):
    state = np.zeros(state_dim)
    episode_return = 0
    for s in tqdm(range(state_dim), desc="loop for e = {}".format(e), leave=False):
        action = agent.take_action(state)
        next_state = state
        for i in range(s, state_dim):
            next_state[i] = 0.1 * action - 1
        reward = -1 * fun2(next_state, *args)
        agent.memorize(state, action, reward, next_state)
        state = next_state
        episode_return += reward
        if len(agent.memory) > batch_size:
            states, actions, rewards, next_states = agent.sample(batch_size)
            done = True if s == state_dim-1 else False
            agent.update(states, actions, rewards, next_states, done)
    e_yield = fun2(state, *args)
    print("episode: {}/{}, score: {}, yield: {}".format(e+1, num_episodes, episode_return, e_yield))
    episode_list.append(episode_return)
    yield_list.append(e_yield)
    state_list.append(state)

## Final obtained magnetic field pulses

In [ ]:
import seaborn as sns
fig, ax = plt.subplots(figsize=(20,8))
sns.lineplot(x=range(len(yield_list)), y=yield_list, marker = "o", markersize = 10)
plt.show()

In [ ]:
state_list[-1]